In [1]:
import nltk 
from nltk.corpus import stopwords
import json
import arff # https://pypi.python.org/pypi/liac-arff
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/Phil/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#object that holds a lot of our data

class NewsObject:
    id = '0'
    targetTitle = []
    targetDescription = ""
    targetKeywords = []
    targetParagraphs = []
    targetCaptions = []
    postText = []
    postMedia = []
    postTimestamp = ''
    #truthMedian = ''
    #truthMean = ''
    #truthMode = ''
    truthClass = ""
    #truthJudgments = []
    attributes = ()
    
    def __init__(self, line):
        
        self.id = line['id']
        self.targetTitle= line['targetTitle']
        self.targetKeywords = line['targetKeywords']
        self.targetParagraphs = line['targetParagraphs']
        self.targetCaptions = line['targetCaptions']
        self.postText = line['postText']
        self.postMedia = line['postMedia']
        self.postTimestamp = line['postTimestamp']
        
    def addTruth(self, line):
        #self.truthMedian = line['truthMedian']
        #self.truthMean = line['truthMean']
        #self.truthMode = line['truthMode']
        if line['truthClass'] == 'clickbait':
            self.truthClass = '1'
        else:
            self.truthClass = '0'
        #self.truthJudgments = line['truthJudgments']
        
    def extractFeatures(self):
        f1 = 0
        f2 = False
        f3 = False
        f4 = 0
        f5 = 0
        f6 = self.truthClass

        words = item.postText[0].split(' ')
        f1 = len(words)
        
        informal = ["you", "you're", "we", "our", "your", "my"]
        questionWords = ['who', 'what', 'where', 'how', 'why']
        stopWords = set(stopwords.words('english'))
        
        
        for word in words:
            if word.lower() in informal:
                f2 = True
            if word.lower() in questionWords:
                f3 = True
        
        text = nltk.word_tokenize(item.postText[0])
        tokenizedList = nltk.pos_tag(text)
        
        
        numProper = 0
        numStop = 0
        for partOfSpeech in tokenizedList:
            if partOfSpeech[1] == 'NNP':
                numProper += 1
            elif(partOfSpeech[0] in stopWords):
                numStop += 1
        
        f4 = round(numStop/f1, 2)
        f5 = round(numProper/f1, 2)
        
        feat = (f1, f2, f3, f4, f5, f6)
        self.attributes = feat
        
        
        
        
        

            

In [3]:
#import files
instances = []

with open('dataset/instances_train.jsonl') as file:
    for line in file:
        temp = NewsObject(json.loads(line))
        instances.append(temp)
        
with open('dataset/truth_train.jsonl') as file2:
    i = 0
    for line in file2:
        instances[i].addTruth(json.loads(line))
        i += 1
#print(instances[0].targetKeywords)

In [5]:
### dump to arff
features = [("word count", 'NUMERIC'),
            ("contains informal pronouns", ['True', 'False']),
            ("contains question word", ['True', 'False']),
            ("percent stop words", 'NUMERIC'),
            ("Percent proper nouns", 'NUMERIC'),
            ("label", ['0', '1'])]
data = {}
data.setdefault('attributes', features)
data.setdefault('description', '')
data.setdefault('relation', 'clickbait_sample')
data.setdefault('data', [])
for item in instances:
    if(item.truthClass == '0'):
        print(item.postText[0])
        print(item.attributes)
    data['data'].append(item.attributes)

with open('sample_train.arff', 'w') as f:
    f.write(arff.dumps(data))

Apple's iOS 9 'App thinning' feature will give your phone's storage a boost
(13, True, False, 0.23, 0.08, '0')
RT @kenbrown12: Emerging market investors are doing their best Monty Pythons--"Run away, run away"
(14, False, False, 0.21, 0.21, '0')
How theme parks like Disney World left the middle class behind
(11, False, True, 0.09, 0.18, '0')
Dez Bryant is reportedly considering skipping Week 1 game if he doesn’t get a long-term deal
(16, False, False, 0.38, 0.12, '0')
Pregnant mother of 12 accused of keeping kids in waste-filled home.
(11, False, False, 0.27, 0.0, '0')
RT @fionamatthias: 10 ways the expat life Is like a continual espresso buzz via @WSJ
(15, False, False, 0.13, 0.27, '0')
House #GOP plans two days of debate, Friday showdown vote on Obama trade agenda.
(14, False, False, 0.14, 0.29, '0')
Azeri government behind foreign media ban, say European Games officials
(10, False, False, 0.0, 0.1, '0')
Only one in three of us complain when we are unhappy with the NHS...
(14, True, 

Michigan teen carries brother for 57 miles to raise awareness of #cerebralpalsy:
(12, False, False, 0.25, 0.08, '0')
JUST IN: Judge backs charges against Cleveland officers in killing of Tamir Rice
(13, False, False, 0.23, 0.38, '0')
Charles Manson prosecutor Vincent Bugliosi dies at 80.
(8, False, False, 0.12, 0.5, '0')
A skyscraper made of wood? Architects design 'world's tallest' wooden skyscraper:
(11, False, False, 0.09, 0.09, '0')
RT @YahooForde: Column: Why American Pharoah should be Sportsman of the Year, Athlete of the Year and win all the ESPYs
(21, False, True, 0.43, 0.38, '0')
RT @BBCSport: The European Games 2015 are open.

Here's a day-by-day guide to @TeamGB's prospects #baku2015 http://
(16, False, False, 0.19, 0.31, '0')
RT @alyssabereznak: morning! i’ll be liveblogging WWDC today alongside @tynanwrites and @Pogue. check us out here
(16, False, False, 0.38, 0.38, '0')
iPad magician strikes again, this time using the Apple Watch
(10, False, False, 0.3, 0.2, '0')
Cartier

Tea Party allies rally on Capitol Hill for ObamaCare repeal   via @josephweber19
(14, False, False, 0.14, 0.43, '0')
A Danish official cuts cake to celebrate an anti-immigration law, but not everyone joins the party
(16, False, False, 0.31, 0.0, '0')
Many concerned about falling temperatures and icy conditions on roads across the Northeast.
(13, False, False, 0.31, 0.08, '0')
One #WBC2017 fan’s attempt to catch a home run ball ended in serious pain 😳

🎥:
(15, False, False, 0.27, 0.13, '0')
Slovenia is honoring its beloved Melania Trump with her own wine
(11, False, False, 0.45, 0.27, '0')
Audi and VW sites raided in emissions probe
(8, False, False, 0.25, 0.25, '0')
Leah Remini says she needs to continue her mission to expose Scientology's dark side
(14, False, False, 0.29, 0.21, '0')
The Ball family’s home was robbed while they were at LaMelo and LiAngelo’s basketball game
(15, False, False, 0.53, 0.33, '0')
Western Mosul reduced to rubble in fight to oust ISIS
(10, False, False, 0.3,

Caitlyn Jenner reveals what her life is like today
(9, False, True, 0.33, 0.22, '0')
Regular bedtimes stop children from becoming overweight, study finds
(9, False, False, 0.11, 0.0, '0')
What markets should conclude from France's election  via @BV
(10, False, True, 0.2, 0.3, '0')
Arrests made in Coachella 2017 cellphone thefts
(7, False, False, 0.14, 0.14, '0')
Taliban insurgents disguised in army uniforms kill up to 140 Afghan soldiers
(12, False, False, 0.25, 0.08, '0')
Meet Zamzam Farah: Olympian, refugee and the most extraordinary athlete at this year's #LondonMarathon
(14, False, False, 0.36, 0.29, '0')
IMF meeting drops anti-protectionism pledge
(5, False, False, 0.0, 0.2, '0')
Family of Aaron Hernandez victim issues "friendly challenge" to Patriots: Pay up
(12, False, False, 0.25, 0.17, '0')
Rare parchment copy of Declaration of Independence found by Harvard researchers
(11, False, False, 0.27, 0.27, '0')
Scientists are developing Star Wars-style technologies to farm drinking w

(9, True, False, 0.0, 0.33, '0')
Who are Trump's 20 million Twitter followers? We investigate:
(9, True, True, 0.11, 0.11, '0')
Maroon 5 &amp; Kendrick Lamar's "Don't Wanna Know" tops the #PopSongs chart
(12, False, False, 0.08, 0.5, '0')
CBS NEWS POLL: More Americans disapprove than approve Trump's handling of transition
(12, False, False, 0.17, 0.33, '0')
Antonio Brown issues apology for Facebook Live post following win over Chiefs
(12, False, False, 0.17, 0.42, '0')
Everything you need to know about the DNC race
(9, True, False, 0.44, 0.11, '0')
11 heartwarming and hilarious stories about LGBT parents coming out to their kids
(13, False, False, 0.38, 0.08, '0')
This new trading app helps investors react to real-time events
(10, False, False, 0.1, 0.0, '0')
7 hidden iPhone tricks that only power users know about
(10, False, False, 0.3, 0.0, '0')
Afghan 'green-eyed girl' Sharbat Gula in quest for a new life
(11, False, False, 0.27, 0.27, '0')
How to know if your account has been hacke

...

(15, False, False, 0.27, 0.07, '0')
Meet the man who has put a thousand souls to #RIP!
(11, False, True, 0.45, 0.18, '0')
Eating a Mediterranean diet could diminish the brain shrinkage that comes with aging
(13, False, False, 0.31, 0.08, '0')
.@beaconband sets the mood with contemplative dance jams at #BillboardLounge
(10, False, False, 0.3, 0.1, '0')
New paintings by one of the world's highest-paid living artists
(10, False, False, 0.3, 0.1, '0')
According to his attorney, Nicki Minaj's brother will fight his rape charges.
(12, False, False, 0.33, 0.17, '0')
7 apps to help you get the most out of Super Bowl Sunday
(13, True, False, 0.46, 0.23, '0')
Meek Mill laughs off rumors linking him to Nicki Minaj’s recent robbery.
(12, False, False, 0.33, 0.42, '0')
.@akshaykumar and @RanveerOfficial sing Hookah Bar
(6, False, False, 0.17, 0.67, '0')
ICYMI: Federal judge issues ruling temporarily halting Trump’s travel ban
(10, False, False, 0.1, 0.3, '0')
A top partner at $20 billion investment

(10, True, False, 0.2, 0.1, '0')
#Cash seizures have witnessed a massive jump across #UP, #Uttarakhand, #Punjab and #Goa
(13, False, False, 0.23, 0.38, '0')
US troops in Iraq operating closer to front lines   via @JenGriffinFNC
(13, False, False, 0.15, 0.23, '0')
#hepatitisC  will be eradicated in Australia by 2026, researchers predict
(11, False, False, 0.36, 0.09, '0')
18 types of men that Ryan Gosling was in the ’00s
(11, False, False, 0.45, 0.18, '0')
Famine declared in South Sudan; 4.9 million people need urgent help
(11, False, False, 0.09, 0.18, '0')
McDonald's workers reveal their VERY clever hacks to save money and get fresher food
(14, False, False, 0.21, 0.14, '0')
Woman appointed as first female head of Saudi Arabia's stock exchange
(11, False, False, 0.18, 0.27, '0')
French Vogue makes history with first transgender cover model
(9, False, False, 0.11, 0.11, '0')
After this man's son killed himself, he decided the world needed a little more kindness
(15, False, False, 0.4, 

Evening Briefing: Here's what you need to know at the end of the day
(14, True, True, 0.5, 0.0, '0')
Adam Schiff slams Devin Nunes over disclosure of Trump wiretap claims
(11, False, False, 0.18, 0.45, '0')
Drake got a Sade tattoo
(5, False, False, 0.2, 0.4, '0')
Conor McGregor's reduced punishment should open the door for him to apply for a professional boxing license.
(17, False, False, 0.41, 0.12, '0')
Ed Sheeran's #ShapeOfYou &amp; 4 other songs that added writers after release
(12, False, False, 0.25, 0.25, '0')
Phife Dawg changed my life, and probably yours too 🙏  by @N_C_B
(13, True, False, 0.38, 0.23, '0')
This is how much worse the world's #watercrisis is getting. #worldwaterday
(11, False, True, 0.36, 0.0, '0')
Trump's net worth has dropped $200 million since his election win
(11, False, False, 0.18, 0.09, '0')
Brexit with no trade deal would cause 'significant damage' to service sector, say Peers - Politics live
(17, False, False, 0.18, 0.0, '0')
Scientists say climate chang

(9, False, False, 0.11, 0.11, '0')
Senators can't say why Trump thinks his Syria strike was legal ➡️ by @lbarronlopez @mmauliff
(15, False, True, 0.27, 0.27, '0')
Venezuela opposition leader Capriles 'banned from politics'
(7, False, False, 0.14, 0.29, '0')
Watch WizKid's kaleidoscopic "Come Closer" video featuring Drake
(8, False, False, 0.0, 0.62, '0')
These are the unhappiest jobs of 2017
(7, False, False, 0.43, 0.0, '0')
Five things you need to know today
•US strikes Syria
•SCOTUS
•Russia investigation
•Baltimore police
•Obamacare
(11, True, False, 0.18, 0.45, '0')
Want to do your bit for #Syria and the Syrians? This is something you could consider
(16, True, False, 0.5, 0.12, '0')
Russians were notified of U.S. airstrikes in Syria prior to missile strikes, U.S. official says.
(15, False, False, 0.27, 0.2, '0')
"How I regularly book round-trip international flights for less than $400"
(11, False, False, 0.18, 0.0, '0')
Opinion: Pepsi shows how America loves to erase black women
(10

(12, False, False, 0.08, 0.42, '0')
How @CarisLeVert, a rookie with the Brooklyn Nets, spends his Sundays
(11, False, True, 0.36, 0.27, '0')
How to make your partner appreciate you more
(8, True, True, 0.5, 0.0, '0')
Right before they gave up, investigators figured out where to look for MH370
(13, False, True, 0.54, 0.08, '0')
Carrie Fisher’s urn is a giant Prozac pill  via @YahooCelebrity
(11, False, False, 0.27, 0.55, '0')
A growing number of parents are repaying their children’s college debt
(11, False, False, 0.36, 0.0, '0')
Washington-area clinic Carafem barred from advertising in Metro stations
(9, False, False, 0.22, 0.22, '0')
Sisters reunite with long-lost mother after 4 decades
(8, False, False, 0.25, 0.0, '0')
New York City's most iconic hotel is closing indefinitely — look back at its famous past
(16, False, False, 0.25, 0.19, '0')
What is President Obama's economic legacy? Here's a look at his jobs record compared with other presidents
(17, False, True, 0.35, 0.12, '0')
.@